# Cleaning of a large NYSE dataset and fuzzy matching on company names from two different datasets.
Francis Peng <br>
BA Economics/Mathematics, Music - May 2023 <br>
Emory University Department of Economics

Parts of the following notebooks are work in progress denoted by the markers **_Begin Work in Progress_** and **_End Work in Progress_**

The following notebook desscribes and executes the process of cleaning a large dataset of NYSE stock listings as well as matching company names from two different datasets. In this process, the rapidfuzz library is used to implement fuzzy matching. Fuzzy matching is needed as the same company may appear differently in the two datasets. For example, the same company might be listed as "X Corporation" in one dataset and "X Corp" in the other. These are the same comapny, and thus should be matched as such.

This fuzzy matching process was developed for the purposes of the following working papers:

*Citation 1*

*Citation 2*

To protect the integrity of the unpublished materials, the source datasets are not output in their entirety anywhere in the following notebook. Only the final result table has been displayed to show that the process does indeed work.

In [179]:
# Installing rapidfuzz.
# !pip install rapidfuzz

In [180]:
# importing needed libraries
import pandas as pd
import numpy as np
from rapidfuzz import process
from rapidfuzz import fuzz

**_Begin Work in Progreess_**

# Cleaning up the large dataset of NYSE listings.
The big dataset contains companies listed on the NYSE (aka CRSP dataset)

The cleaning of this dataset aims to achieve the following: <br>
1. Standardize the naming conventions of companies.
2. Fill in company names in the 'comnam' column using the 'permco' column.

In [181]:
# Load the dataset
nodup = pd.read_stata('full_dataset.dta')

In [182]:
# big_df.head() # taking a look

In [183]:
# len(big_df) # rows total

In [184]:
# print(len(pd.unique(big_df['comnam']))) # unique company names
# print(len(pd.unique(big_df['permco']))) # unique permco
# We are done when # unique comnam = # unique permco

In [185]:
# nodup = big_df.drop_duplicates(subset=['comnam', 'permco']) # remove duplicates, keep unique in comnam and permco fields

In [186]:
print(len(pd.unique(nodup['comnam']))) # unique company names
print(len(pd.unique(nodup['permco']))) # unique permco

2758
1574


In [187]:
nodup.head() # taking a look

,ipo_date,crsp_date,permno,Date,date,comnam,prc,vol,month,day,year,numdate,permco,siccd
0,1946-09-19,1925-12-31,10006.0,9/19/1946,19320804,,9.250,2200.0,NaN,NaN,NaN,NaT,22156.0,3740.0
1,1949-09-23,1948-01-16,10137.0,9/23/1949,19390607,,10.000,3200.0,NaN,NaN,NaN,NaT,20045.0,4910.0
2,1937-05-01,1954-05-03,10321.0,5/1/1937,19420204,,4.625,2800.0,NaN,NaN,NaN,NaT,22184.0,3710.0
3,1937-04-15,1925-12-31,10487.0,4/15/1937,,Am. Steel Foundries,66.875,1100.0,12.0,1.0,1916.0,1916-12-01,22195.0,3310.0
4,1936-04-09,1925-12-31,10604.0,1/9/1946,19461026,,35.500,200.0,NaN,NaN,NaN,NaT,20227.0,2910.0


In [188]:
# Cleaning up company names
nodup['comnam'] = nodup['comnam'].str.upper()
nodup['comnam'] = nodup['comnam'].replace(
    regex={',':'', '\.':'', '-':' '}).replace(
    regex={'CORPORATION':'CO', 'INCORPORATED':'INC', 'COMPANY':'CO', 'LIMITED':'LTD'}
)

In [189]:
print(len(pd.unique(nodup['comnam']))) 
print(len(pd.unique(nodup['permco'])))
print(len(nodup))

2508
1574
7740460


In [190]:
# test = nodup.dropna(subset=['permco'])
# print(len(pd.unique(test['permco'])))
# print(len(test))
# print(len(pd.unique(test['comnam'])))

\# of unique comnam reduced without reducing # of permco which is good <br>

Next, if it is true that all permco have at least 1 row where the comnam is present, then removing rows where comnam is an empty string should not remove any permcos.

In [191]:
nodup['comnam'].replace('', np.nan, inplace = True) # replacing empty strings with null values
noblank = nodup.dropna(subset=['comnam']) # dropping null values

In [192]:
print(len(pd.unique(noblank['permco'])))

987


From the above output, we find that there are in fact permcos where no row exists where there is a comnam, i.e., some permcos are unidentifiable with comnam. The code directly below gives us the number of such permcos and the specific permcos that are unidentifiable.

In [193]:
a = pd.unique(noblank['permco'])
b = pd.unique(nodup['permco'])
print(len(np.setdiff1d(b, a)))
c = np.asarray(np.setdiff1d(b, a))
np.savetxt("permco_missing_comnam.csv", c, delimiter=",")

588


In [194]:
nodup[nodup['permco'] == 22195].head()

,ipo_date,crsp_date,permno,Date,date,comnam,prc,vol,month,day,year,numdate,permco,siccd
3,1937-04-15,1925-12-31,10487.0,4/15/1937,,AM STEEL FOUNDRIES,66.875,1100.0,12.0,1.0,1916.0,1916-12-01,22195.0,3310.0
722,NaT,NaT,1020.0,,,AM STEEL FDRIES,93.000,NaN,11.0,14.0,1919.0,1919-11-14,22195.0,3310.0
723,NaT,NaT,1020.0,,,AM STEEL F'DRIES,94.250,NaN,11.0,7.0,1919.0,1919-11-07,22195.0,3310.0
724,NaT,NaT,1020.0,,,AM STEEL F?DRIES,95.000,NaN,10.0,24.0,1919.0,1919-10-24,22195.0,3310.0
725,NaT,NaT,1030.0,,,AM STEEL FOUNDRIES,94.250,NaN,11.0,7.0,1919.0,1919-11-07,22195.0,3310.0


In [195]:
pd.unique(nodup.loc[nodup['permco'] == 56225, 'comnam'])

array([nan], dtype=object)

The above code confirms what was previously mentioned with the specific permco 21394. There does not exist a row in the dataset for permco 21394 where a comnam is present, thus it is unidentifiable.

Regardless of this result which will be dealt with later, we continue with the cleaning. In particular we now want to fill in comnam for permcos that are identifiable.

1. First, we create a dataframe from the existing dataframe such that we have only comnam and permco.
2. Then, we use this dataframe and join it with the original dataframe on permco.

In [196]:
# Step 1.
# Create a dataframe with only comnam and permco
working_df = nodup[['comnam', 'permco']].copy()

In [197]:
working_df.dropna(subset = ['comnam'], inplace = True)
working_df.dropna(subset = ['permco'], inplace = True)

In [198]:
working_df.drop_duplicates(subset = ['comnam'], inplace = True)
working_df

,comnam,permco
3,AM STEEL FOUNDRIES,22195.0
7,COMMERCIAL INVESTMENT TRUST CO,22239.0
18,GENERAL MOTORS CORP,20799.0
23,INGERSOLL RAND CO,20977.0
24,INLAND STEEL CO,20978.0
...,...,...
7739263,MIXICAN PETROLEUM,14000.0
7740114,MINNESOTA & ST LOUIS,23113.0
7740134,ALBANY & SUS,10100.0
7740255,PAN AMER PETROLEUM,22466.0


In [199]:
working_df = working_df.sort_values('comnam', key = lambda x:x.str.len(), ascending = False)

In [200]:
working_df = working_df.drop_duplicates(subset = ['permco'], keep = 'first' )

In [201]:
test1 = working_df.reset_index(drop = True)
test1

,comnam,permco
0,NEWPORT NEWS & HAMPTON RY GAS & ELECTRIC,22445.0
1,PITTSBURGH & WEST VIRGINIA RY CO,21413.0
2,ATCHISON TOPEKA & SANTA FE RY CO,22204.0
3,NATIONAL ENAMELING & STAMPING CO,22433.0
4,DULUTH SOUTH SHORE & ATLANTIC RY,22299.0
...,...,...
979,NY STEAM,14460.0
980,COTY INC,22283.0
981,AM MALT,10360.0
982,RUTLAND,15600.0


In [202]:
len(test1)

984

In [203]:
test1 = test1.rename(columns={'comnam':'new_name'})
new_df = nodup.join(test1.set_index('permco'), on='permco')

In [220]:
new_df.to_csv('full_dataset_with_comnam.csv')

**_End Work in Progress_**

# Fuzzy Matching

In [210]:
# Loading the two datasets
dataA = pd.read_excel('Data.xlsx')
# dataB = pd.read_excel('CRSP.xlsx')
dataB = new_df.copy()

### Cleaning Up the Data

In [211]:
dataB = dataB.rename(columns={'new_name':'company'})
dataB.head()

,ipo_date,crsp_date,permno,Date,date,comnam,prc,vol,month,day,year,numdate,permco,siccd,company
0,1946-09-19,1925-12-31,10006.0,9/19/1946,19320804,NaN,9.250,2200.0,NaN,NaN,NaN,NaT,22156.0,3740.0,AMERICAN CAR & FDRY CO
1,1949-09-23,1948-01-16,10137.0,9/23/1949,19390607,NaN,10.000,3200.0,NaN,NaN,NaN,NaT,20045.0,4910.0,AMERICAN WATER WORKS & ELEC INC
2,1937-05-01,1954-05-03,10321.0,5/1/1937,19420204,NaN,4.625,2800.0,NaN,NaN,NaN,NaT,22184.0,3710.0,NASH MOTORS CO
3,1937-04-15,1925-12-31,10487.0,4/15/1937,,AM STEEL FOUNDRIES,66.875,1100.0,12.0,1.0,1916.0,1916-12-01,22195.0,3310.0,AMERICAN STEEL FOUNDRIES
4,1936-04-09,1925-12-31,10604.0,1/9/1946,19461026,NaN,35.500,200.0,NaN,NaN,NaN,NaT,20227.0,2910.0,ATLANTIC REFNG CO


In [212]:
# Dropping duplicates
dataAW = dataA.dropna(subset = ['Company Name']).drop_duplicates('Company Name', keep = 'first').reset_index(drop=True)

In [213]:
dataBW = dataB.dropna(subset = ['company']).drop_duplicates('company', keep = 'first').reset_index(drop=True)

In [214]:
# Replacing common terms with abbreviations, deleting punctuation, making everything uppercase for ease of reading.
dataAW['N_Company Name'] = dataAW['Company Name'].str.upper()
dataBW['N_company'] = dataBW['company'].str.upper()

dataAW['N_Company Name'] = dataAW['N_Company Name'].replace(
    regex={',':'', '\.':'', '-':' '}).replace(
    regex={'CORPORATION':'CO', 'INCORPORATED':'INC', 'COMPANY':'CO', 'LIMITED':'LTD'}
)

dataBW['N_company'] = dataBW['N_company'].replace(
    regex={',':'', '\.':'', '-':' '}).replace(
    regex={'CORPORATION':'CO', 'INCORPORATED':'INC', 'COMPANY':'CO', 'LIMITED':'LTD'}
)

#dataAW['N_Company Name'] = dataAW['N_Company Name'].replace(
#    regex={',':'', '\.':'', '-':' '}).replace(
#    regex={'CORPORATION':'', 'INCORPORATED':'', 'COMPANY':''}).replace(
#    regex={'LTD':'', 'CORP':'', 'INC':''}).replace(
#    regex={'CO':''}
#)

#dataBW['N_company'] = dataBW['N_company'].replace(
#    regex={',':'', '\.':'', '-':' '}).replace(
#    regex={'CORPORATION':'', 'INCORPORATED':'', 'COMPANY':''}).replace(
#    regex={'LTD':'', 'CORP':'', 'INC':''}).replace(
#    regex={'CO':''}
#)


dataAW = dataAW.dropna(subset = ['N_Company Name']).drop_duplicates('N_Company Name', keep = 'first').reset_index(drop=True)
dataBW = dataBW.dropna(subset = ['N_company']).drop_duplicates('N_company', keep = 'first').reset_index(drop=True)

In [215]:
dataBW

,ipo_date,crsp_date,permno,Date,date,comnam,prc,vol,month,day,year,numdate,permco,siccd,company,N_company
0,1946-09-19,1925-12-31,10006.0,9/19/1946,19320804,NaN,9.250,2200.0,NaN,NaN,NaN,NaT,22156.0,3740.0,AMERICAN CAR & FDRY CO,AMERICAN CAR & FDRY CO
1,1949-09-23,1948-01-16,10137.0,9/23/1949,19390607,NaN,10.000,3200.0,NaN,NaN,NaN,NaT,20045.0,4910.0,AMERICAN WATER WORKS & ELEC INC,AMERICAN WATER WORKS & ELEC INC
2,1937-05-01,1954-05-03,10321.0,5/1/1937,19420204,NaN,4.625,2800.0,NaN,NaN,NaN,NaT,22184.0,3710.0,NASH MOTORS CO,NASH MOTORS CO
3,1937-04-15,1925-12-31,10487.0,4/15/1937,,AM STEEL FOUNDRIES,66.875,1100.0,12.0,1.0,1916.0,1916-12-01,22195.0,3310.0,AMERICAN STEEL FOUNDRIES,AMERICAN STEEL FOUNDRIES
4,1936-04-09,1925-12-31,10604.0,1/9/1946,19461026,NaN,35.500,200.0,NaN,NaN,NaN,NaT,20227.0,2910.0,ATLANTIC REFNG CO,ATLANTIC REFNG CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,NaT,NaT,NaN,,,FEDERAL SUGAR REFINING,45.000,100.0,3.0,10.0,1911.0,1911-03-10,12350.0,2060.0,FEDERAL SUGAR REFINING,FEDERAL SUGAR REFINING
980,NaT,NaT,NaN,,,R R I & P,31.875,18000.0,11.0,26.0,1920.0,1920-11-26,15400.0,4010.0,R R I & P,R R I & P
981,NaT,NaT,NaN,,,LAKE SHORE,350.000,9.0,12.0,8.0,1911.0,1911-12-08,13550.0,4010.0,LAKE SHORE,LAKE SHORE
982,NaT,NaT,NaN,,,INTERBORO COPPER,8.375,100.0,6.0,14.0,1918.0,1918-06-14,13210.0,3331.0,INTERBORO COPPER,INTERBORO COPPER


In [216]:
# Matching using the fuzzywuzzy library
results_df = pd.DataFrame(columns = ['Match_Score', 'N_AWname', 'N_BWName', 'AWName'])
for index, row in dataAW.iterrows():
    result = process.extractOne(row['N_Company Name'], dataBW['N_company'])
    #if result[1] >= 86:
    results_df = results_df.append({'Match_Score':result[1], 'N_AWname':row['N_Company Name'], 'N_BWName':result[0], 'AWname':row['Company Name']}, ignore_index=True)

In [217]:
# Sorting values by match score given by fuzzywuzzy
results_final = results_df.sort_values(by = ['Match_Score'], ascending = False).drop_duplicates('N_AWname', keep = 'first').reset_index(drop=True)

In [218]:
results_final

,Match_Score,N_AWname,N_BWName,AWName,AWname
0,100.000000,GENERAL ICE CREAM CORP,GENERAL ICE CREAM CORP,NaN,General Ice Cream Corp.
1,100.000000,GREAT NORTHERN RAILWAY CO,GREAT NORTHERN RAILWAY CO,NaN,Great Northern Railway Company
2,100.000000,GENERAL MILLS INC,GENERAL MILLS INC,NaN,"General Mills, Inc."
3,100.000000,NATIONAL BISCUIT CO,NATIONAL BISCUIT CO,NaN,National Biscuit Company
4,100.000000,SUN OIL CO,SUN OIL CO,NaN,Sun Oil Co.
...,...,...,...,...,...
6471,54.000000,CREOLE SYNDICATE,LOEW'S INC,NaN,Creole Syndicate
6472,54.000000,PICRADILLVREALTYCOINDIANAPOLIS,PACIFIC MAIL,NaN,"PicradillvRealtyCo.,Indianapolis,"
6473,54.000000,MONROE LOAN SOCIETY,LAKE SHORE,NaN,Monroe Loan Society
6474,51.428571,WILLOW BROOK DAIRY,THE FAIR,NaN,Willow Brook Dairy


In [219]:
# Writing to a csv file
output = results_final
output.to_csv('10_27_rapidfuzz.csv')